In [15]:
%matplotlib inline
%cd ~/nbs
import sys, os
from PIL import Image
import numpy as np
sys.path.append(os.path.join(os.getcwd(), "part2_orig"))
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
from keras import metrics
import os
from vgg16_avg import VGG16_Avg


/home/ubuntu/nbs


In [16]:
# Tell Tensorflow to use no more GPU RAM than necessary
# limit_mem()

In [17]:
pwd = os.getcwd()
print(pwd)

path = pwd + '/data/imagenet/sample/'
dpath = pwd + '/data/'
resultsPath = pwd + '/data/output/'
print(path)
print(dpath)

/home/ubuntu/nbs
/home/ubuntu/nbs/data/imagenet/sample/
/home/ubuntu/nbs/data/


In [18]:
def image_preprocess(img_ar):
    '''
    Input: image as numpy array
    Output: preprocessed image as numpy array
    '''
    resnet_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
    exp_img_ar = np.expand_dims(np.array(img_ar), 0)
    proc_img_ar = (exp_img_ar - resnet_mean)[:,:,:,::-1]
    return proc_img_ar
#end

def image_postprocess(img_ar, shp):
    '''Input: preprocessed image as numpy array
       Output: postprocessed image as numpy array
    '''
    resnet_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
    postpr_img_ar = np.clip(img_ar.reshape(shp)[:,:,:,::-1] + resnet_mean, 0, 255)
    return postpr_img_ar
#end

In [19]:
resize_to = (500, 350)

vangogh_style = Image.open(dpath + 'munch.jpeg').resize(resize_to)
leaningtower_content = Image.open(dpath + 'trump.jpg').resize(resize_to)

vangogh_style_ar = image_preprocess(vangogh_style)
leaningtower_content_ar = image_preprocess(leaningtower_content)

shp = vangogh_style_ar.shape

In [20]:
def vgg_avgpooling(vgg_model):
    vgg_avg_model = Sequential()
    for i, layer in enumerate(vgg_model.layers):
        name = layer.name
        if type(layer)!=MaxPooling2D:
            vgg_avg_model.add(layer)
        else:
            vgg_avg_model.add(AveragePooling2D((2, 2), strides=(2, 2), name=name[0:6] + '_avgpool'))
        #end
    #end
    return vgg_avg_model
#end

In [21]:
def content_loss(computed, target, weight_ls=None):
    '''
    Input: computed and target tensors (or lists of tensors for more than one content layer)
    Output: content loss calculated as MSE and scaled by the tensor(s) dimension
    '''
    if isinstance(computed, list):
        if not weight_ls:
            weight_ls = [1.0 for layer in computed]
        #end
        c_loss = sum([K.sum(metrics.mse(comp[0], targ[0]) * w \
                      for comp, targ, w in zip(computed, target, weight_ls))])
        _, height, width, channels = map(lambda i: i, K.int_shape(computed[0]))
    else:
        c_loss = K.sum(metrics.mse(computed, target))
        _, height, width, channels = K.int_shape(computed)
    #end
    c_loss = c_loss #/ (height * width * channels)
    return c_loss
#end

In [22]:
def gram_matrix(tens):
    features = K.batch_flatten(K.permute_dimensions(tens, (2, 0, 1))) # rows = channels, columns = flattened x, y
    gram = K.dot(features, K.transpose(features)) / tens.get_shape().num_elements() #correlate each pair of channels
    return gram
#end

def style_loss(computed, target, weight_ls=None):
    '''
    Input: computed and target tensors (or lists of tensors for more than one style layer)
    Output: content loss calculated as MSE of the Gram matrices and scaled by the tensor(s) dimension
    '''
    if isinstance(computed, list):
        if not weight_ls:
            weight_ls = [1.0 for layer in computed]
        #end
        s_loss = sum([K.sum(metrics.mse(gram_matrix(comp[0]), gram_matrix(targ[0]))) * w \
                      for comp, targ, w in zip(computed, target, weight_ls)])
        _, height, width, channels = map(lambda i: i, K.int_shape(computed[0]))
    else:
        s_loss = K.sum(metrics.mse(gram_matrix(computed), gram_matrix(target)))
        _, height, width, channels = K.int_shape(computed)
    #end
    s_loss = s_loss #/ (height * width * channels)
    return s_loss
#end

In [23]:
def total_loss(style_layer_ls, style_targ_ls, style_wgt_ls, content_layer, content_targ, style2content_ratio):
    s_loss = style_loss(style_layer_ls, style_targ_ls, style_wgt_ls)
    c_loss = content_loss(content_layer, content_targ)
    loss = s_loss + c_loss / style2content_ratio
    return loss
#end


In [24]:
def get_content_targets(style_transfer_model, content_ar):
    '''
    Input: style transfer model and numpy array of the content image
    Output: the output of the model at the content layer and its respective target
    '''
    layer_output_dc = {l.name: l.get_output_at(0) for l in style_transfer_model.layers}
    content_layer = layer_output_dc['block4_conv2'] #change it to another layer of choice if necessary
    content_model = Model(style_transfer_model.input, content_layer)
    content_targ = K.variable(content_model.predict(content_ar))
    return content_layer, content_targ
#end 

def get_style_targets(style_transfer_model, style_ar):
    '''Input: style transfer model and numpy array of the style image
       Output: the output of the model at the style layer(s) and its respective target
    '''
    layer_output_dc = {l.name: l.get_output_at(0) for l in style_transfer_model.layers}
    style_layer_ls = [layer_output_dc['block{}_conv2'.format(o)] for o in range(1,6)] #change it different layers if necessary
    style_model = Model(style_transfer_model.input, style_layer_ls)
    style_targ_ls = [K.variable(o) for o in style_model.predict(style_ar)]
    return style_layer_ls, style_targ_ls
#end 

In [25]:
class Evaluator(object):
    '''
    Initialization: function and shape of the image array
    Returns the loss and the gradients as computed with
       respect to the image that is fed to the CNN
    '''
    def __init__(self, f, shp):
        self.f = f
        self.shp = shp
        return
    #end        
    def loss(self, x):
        loss_, self.grad_values = self.f([x.reshape(self.shp)])
        return loss_.astype(np.float64)
    #end
    def grads(self, x): 
        return self.grad_values.flatten().astype(np.float64)
    #end    
#end

def apply_transfer(eval_obj, n_iter, img, shp, pref='', save=True, verbose=True):
    '''
    Input: evaluator, number of iterations, input image and shape
    Output: final image, list of losses and info dictionary of optimization procedure
    '''
    info_dc = dict()
    loss_ls = list()
    for it in range(n_iter):
        img, min_val, iter_dc = fmin_l_bfgs_b(eval_obj.loss, img.flatten(),
                                              fprime=eval_obj.grads, maxfun=20)
        img = np.clip(img, -127, 127)
        info_dc['iteration_'+str(it+1)] = iter_dc
        loss_ls = loss_ls + [min_val]
        if verbose:
            print('Current loss value:', min_val)
        #end
        if save:
            imsave(resultsPath + pref + 'res_at_iteration_' + str(it+1) + '.png', image_postprocess(img.copy(), shp)[0])
        #end
    #end
    return img, loss_ls, info_dc
#end

In [26]:
iterations = 10

st_transf_model = vgg_avgpooling(VGG16(include_top=False, input_shape=shp[1:]))

content_layer, content_targ = get_content_targets(st_transf_model, leaningtower_content_ar)
style_layer_ls, style_targ_ls = get_style_targets(st_transf_model, vangogh_style_ar)

style_wgt_ls = [0.05,0.2,0.2,0.25,0.3]
style2content_ratio = 15.0
loss = total_loss(style_layer_ls, style_targ_ls, style_wgt_ls, content_layer, content_targ, style2content_ratio)
grads = K.gradients(loss, st_transf_model.input)
transfer_fn = K.function([st_transf_model.input], [loss] + grads)
evaluator = Evaluator(transfer_fn, shp)

virgin_img = np.random.uniform(-2.5, 2.5, shp)/100

image, t_loss, info_dc = apply_transfer(evaluator, iterations, virgin_img, shp)



Current loss value: 6514186.0
Current loss value: 3456853.25
Current loss value: 2664727.5
Current loss value: 2318071.5
Current loss value: 2131818.0
Current loss value: 2010079.0
Current loss value: 1925805.75
Current loss value: 1859138.25
Current loss value: 1804408.25
Current loss value: 1760254.75
